Comprehensive Evaluation of Model Fairness Using Multiple Fairness Metrics

In [1]:
import pandas as pd

df = pd.read_csv('synthetic_stroke_data.csv')

display(df.head(10))
 

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,Female,80.3,0,0,Yes,Govt_job,Urban,170.38,18.4,Unknown,0
1,2,Female,89.4,0,0,Yes,Self-employed,Urban,186.89,19.6,formerly smoked,0
2,3,Male,87.5,0,0,Yes,Children,Urban,118.42,34.7,never smoked,0
3,4,Male,19.7,0,0,Yes,Private,Urban,226.01,27.1,smokes,0
4,5,Male,85.2,0,0,Yes,Children,Rural,229.64,27.6,Unknown,0
5,6,Male,51.0,0,0,Yes,Private,Rural,127.65,25.0,Unknown,0
6,7,Female,73.7,0,0,Yes,Govt_job,Urban,171.32,39.9,smokes,0
7,8,Male,23.7,1,1,No,Private,Urban,163.56,24.3,never smoked,0
8,9,Male,33.4,0,0,Yes,Govt_job,Rural,125.28,30.7,smokes,0
9,10,Female,71.8,0,0,Yes,Govt_job,Urban,177.63,21.1,smokes,0


# Task
Evaluate the fairness of a Logistic Regression model trained on the stroke prediction dataset using Disparate Impact Ratio (DIR), Equal Opportunity Difference (EOD), and Predictive Parity, considering 'gender' as the sensitive attribute. The evaluation should include data cleaning and preprocessing steps.

## Clean and preprocess the data

### Subtask:
Handle missing values, encode categorical features, and scale numerical features.


**Reasoning**:
Handle missing values, encode categorical features, and scale numerical features as per the instructions.



**Reasoning**:
The previous code failed because both `get_feature_names_in` and `get_feature_names` are not working. It seems the scikit-learn version is old. I will try to use `get_feature_names` without arguments, which is the behavior in older versions.



In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# First, split the data into features and target
X = df.drop(['stroke'], axis=1)
y = df['stroke']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identify missing values
print("Missing values before handling:")
display(X_train.isnull().sum())
display(X_test.isnull().sum())

# Handle missing values in 'bmi' using the mean
imputer = SimpleImputer(strategy='mean')
X_train['bmi'] = imputer.fit_transform(X_train[['bmi']])
X_test['bmi'] = imputer.transform(X_test[['bmi']])

print("\nMissing values after handling:")
display(X_train.isnull().sum())
display(X_test.isnull().sum())

# Identify categorical and numerical features
categorical_features = X_train.select_dtypes(include=['object']).columns
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.drop('id') # Exclude 'id'

# Create preprocessing pipelines for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Create a column transformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Keep other columns (like 'id')
)

# Apply the preprocessing to training and testing data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Since getting feature names from OneHotEncoder is problematic, we will create the dataframes directly
# The column order should be numerical features followed by one-hot encoded features and then remainder columns
# We will rely on the order of columns provided to the ColumnTransformer

# Get the number of numerical and categorical features after transformation
num_numerical_features = len(numerical_features)
# The number of one-hot encoded features depends on the unique values in categorical features
# We can get the number of output features from the one-hot encoder
num_categorical_features_processed = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features).shape[0]

 

# Create a list of placeholder column names for the processed dataframe
# This is a workaround since we cannot reliably get the actual feature names
processed_column_names = [f'feature_{i}' for i in range(X_train_processed.shape[1])]


X_train_processed = pd.DataFrame(X_train_processed, columns=processed_column_names, index=X_train.index)
X_test_processed = pd.DataFrame(X_test_processed, columns=processed_column_names, index=X_test.index)


print("\nProcessed training data:")
display(X_train_processed.head())
print("\nProcessed testing data:")
display(X_test_processed.head())

 

Missing values before handling:


id                      0
gender                  0
age                     0
hypertension            0
heart_disease           0
ever_married            0
work_type               0
Residence_type          0
avg_glucose_level       0
bmi                  2018
smoking_status          0
dtype: int64

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  482
smoking_status         0
dtype: int64


Missing values after handling:


id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64


Processed training data:


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
21455,-1.618739,2.369568,-0.336012,0.170863,-1.444052,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,21456.0
16030,-0.417339,-0.422018,-0.336012,-0.786165,-1.358684,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,16031.0
33815,0.370779,-0.422018,-0.336012,0.594901,-1.742841,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,33816.0
25345,-1.488988,2.369568,-0.336012,-1.612452,-1.230631,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,25346.0
28208,0.486113,-0.422018,-0.336012,-1.080813,-0.320035,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,28209.0



Processed testing data:


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
1195,-1.032456,-0.422018,-0.336012,0.741068,-5.054824e-16,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1196.0
8748,-0.556702,2.369568,-0.336012,0.752831,1.387332e+00,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,8749.0
10400,0.538975,2.369568,-0.336012,1.611514,-1.059894e+00,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,10401.0
31269,1.211759,-0.422018,-0.336012,-0.814126,-1.372912e+00,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,31270.0
38533,0.615864,-0.422018,-0.336012,0.400719,-2.062106e-01,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,38534.0


## Split the dataset

### Subtask:
Split the cleaned and preprocessed dataset into training and testing sets, ensuring that the target variable is handled appropriately.


## Train a logistic regression model

### Subtask:
Train a Logistic Regression classification model on the training data.


**Reasoning**:
Train a Logistic Regression model using the processed training data.



In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression model
model = LogisticRegression(solver='liblinear', random_state=42) # Added solver and random_state for reproducibility

# Fit the model to the processed training data
model.fit(X_train_processed, y_train)

 

LogisticRegression(random_state=42, solver='liblinear')

## Predict outcomes

### Subtask:
Predict outcomes on the test data and store the predicted labels.


**Reasoning**:
Use the trained model to make predictions on the processed test data.



In [ ]:
y_pred = model.predict(X_test_processed)

 

## Identify sensitive attribute

### Subtask:
Identify the sensitive attribute for fairness evaluation (e.g., 'gender').


**Reasoning**:
Extract the 'gender' column from the original X_test DataFrame to identify the sensitive attribute for fairness evaluation.



In [ ]:
# Identify the column in the original X_test DataFrame that represents the 'gender' attribute.
# Extract the 'gender' column from the original X_test DataFrame and store it in a variable named sensitive_attribute_test.
sensitive_attribute_test = X_test['gender']

display(sensitive_attribute_test.head())

 

,gender
33553,Female
9427,Male
199,Male
12447,Female
39489,Female


## Calculate fairness metrics

### Subtask:
Calculate the following fairness metrics: Disparate Impact Ratio (DIR), Equal Opportunity Difference (EOD), and Predictive Parity.


**Reasoning**:
Convert the true labels, predicted labels, and sensitive attribute to NumPy arrays and import necessary fairness metrics functions. Define privileged and unprivileged groups.



In [ ]:
%pip install aif360

**Reasoning**:
Now that aif360 is installed, convert the true labels, predicted labels, and sensitive attribute to NumPy arrays and import necessary fairness metrics functions. Define privileged and unprivileged groups and calculate the fairness metrics.



**Reasoning**:
The previous command failed because the `aif360.datasets.BinaryLabelDataset` expects all columns in the input DataFrame `df` to be numerical. The 'gender' column is still in string format ('Male', 'Female'). I need to encode the 'gender' column numerically before creating the `BinaryLabelDataset`.



In [ ]:
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Convert to numpy arrays
y_test_np = y_test.to_numpy()
y_pred_np = y_pred
sensitive_attribute_test_np = sensitive_attribute_test.to_numpy()

# Encode the sensitive attribute 'gender' numerically
# Use LabelEncoder for binary categorical feature
le = LabelEncoder()
sensitive_attribute_test_encoded = le.fit_transform(sensitive_attribute_test_np)

# Define privileged and unprivileged groups using the encoded values
# Assuming 'Male' was encoded to 1 and 'Female' to 0 based on alphabetical order by default
# Let's check the mapping to be sure
gender_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(f"Gender encoding mapping: {gender_mapping}")

# Based on the typical alphabetical encoding, 'Female' (0) is unprivileged and 'Male' (1) is privileged
privileged_groups_encoded = [{'gender': gender_mapping['Male']}]
unprivileged_groups_encoded = [{'gender': gender_mapping['Female']}]

# Define favorable and unfavorable outcomes for the target variable.
favorable_label = 1.0  # Assuming 1 is the favorable outcome
unfavorable_label = 0.0 # Assuming 0 is the unfavorable outcome

# Create a temporary DataFrame for aif360 with the encoded sensitive attribute
aif_df_encoded = pd.DataFrame({
    'gender': sensitive_attribute_test_encoded,
    'true_label': y_test_np,
    'predicted_label': y_pred_np
})

# Create a BinaryLabelDataset for calculating metrics that compare true and predicted labels
# Use 'true_label' as the label_names and 'predicted_label' as scores_names
aif_data_for_metrics = BinaryLabelDataset(
    df=aif_df_encoded,
    label_names=['true_label'],
    protected_attribute_names=['gender'],
    favorable_label=favorable_label,
    unfavorable_label=unfavorable_label,
    scores_names=['predicted_label'] # Specify the column containing predicted labels
)


# Create a ClassificationMetric object
metric_classification = ClassificationMetric(aif_data_for_metrics,
                                             aif_data_for_metrics, # Pass the same dataset for predictions
                                             privileged_groups=privileged_groups_encoded,
                                             unprivileged_groups=unprivileged_groups_encoded)

 

# Calculate Disparate Impact Ratio (DIR)
# DIR is calculated based on the predicted labels only.
# We can use the BinaryLabelDatasetMetric for this, or the ClassificationMetric.
# Let's stick to ClassificationMetric for consistency when comparing with EOD and Predictive Parity.
# Disparate Impact is the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group.
# Rate of favorable outcome is calculated from the 'scores' column (predicted labels).
dir_score = metric_classification.disparate_impact()


# Calculate Equal Opportunity Difference (EOD)
# EOD is the difference in True Positive Rate (TPR) between unprivileged and privileged groups.
eod_score = metric_classification.equal_opportunity_difference()

# Calculate Predictive Parity (Predictive Value Parity)
# Predictive Parity is the difference in Positive Predictive Value (PPV) between unprivileged and privileged groups.
# The method for this is positive_predictive_value
predictive_parity_score = metric_classification.positive_predictive_value(privileged=False) - metric_classification.positive_predictive_value(privileged=True)


print(f"Disparate Impact Ratio (DIR): {dir_score}")
print(f"Equal Opportunity Difference (EOD): {eod_score}")
print(f"Predictive Parity: {predictive_parity_score}")

 

Gender encoding mapping: {'Female': np.int64(0), 'Male': np.int64(1)}
Disparate Impact Ratio (DIR): 0.9405261622589082
Equal Opportunity Difference (EOD): 0.0
Predictive Parity: 0.0


## Summarize findings

### Subtask:
Summarize the results of the fairness evaluation based on the calculated metrics.

**Reasoning**:
Summarize the calculated Disparate Impact Ratio (DIR), Equal Opportunity Difference (EOD), and Predictive Parity scores and interpret what they suggest about the fairness of the model with respect to the 'gender' sensitive attribute.

## Summary of Fairness Evaluation:

Based on the calculated fairness metrics for the Logistic Regression model with 'gender' as the sensitive attribute:

- **Disparate Impact Ratio (DIR):** A DIR of approximately 0.94 suggests that the rate of favorable outcomes (stroke prediction) is roughly the same for the unprivileged group ('Female') compared to the privileged group ('Male'). A DIR closer to 1 generally indicates less disparate impact. In this case, the value is close to 1, suggesting minimal disparate impact based on this metric.

- **Equal Opportunity Difference (EOD):** An EOD of 0.0 indicates that the True Positive Rate (TPR) is the same for both the unprivileged and privileged groups. TPR is the proportion of actual positive cases (strokes) that were correctly identified by the model. An EOD of 0 suggests that the model is equally effective at identifying positive cases for both genders, satisfying the equal opportunity criterion.

- **Predictive Parity:** A Predictive Parity score of 0.0 indicates that the Positive Predictive Value (PPV) is the same for both the unprivileged and privileged groups. PPV is the proportion of positive predictions that were actually correct. A Predictive Parity of 0 suggests that when the model predicts a stroke, it is equally likely to be correct for both genders, satisfying the predictive parity criterion.

**Overall Interpretation:**

Based on the calculated metrics (DIR, EOD, and Predictive Parity all being close to or exactly 1 or 0 as desired for fairness), the Logistic Regression model appears to be relatively fair with respect to the 'gender' attribute on this test dataset, according to these specific definitions of fairness.

It's important to note that fairness is a multifaceted concept, and evaluating it thoroughly often requires considering multiple metrics and potential biases in the data itself.